In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import json
import pandas as pd
import boto3

## Extração

In [ ]:


# 1. Configurações iniciais 

url = 'https://stats.nba.com/players/traditional/?PerMode=Totals&Season=2019-20&SeasonType=Regular%20Season&sort=PLAYER_NAME&dir=-1'
option = Options()
option.headless = True
driver = webdriver.Firefox()
driver.get(url)


    # Rolar a página até o elemento ficar visivel
elemento = driver.find_element(By.CLASS_NAME, "Block_blockContent__6iJ_n")
driver.execute_script("arguments[0].scrollIntoView(true);", elemento)

# 2. Acesar o DropDown para exibir todos os resultados da table que quero recuperar
element = WebDriverWait(driver, 25).until(
    EC.visibility_of_element_located((By.XPATH, "//*[@id='__next']/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select")))

# 2.1 Localizar todos os elementos filhos da dropdown
elements = element.find_elements(By.TAG_NAME,'option')

# 2.2. Selecionar o segundo elemento da dropdown (ALL)
second_element = elements[0]
second_element.click()

# 3. Procurar a Table de interesse com o option do Dropdown selecionado
element = driver.find_element(By.CLASS_NAME, "Crom_base__f0niE")

# 3.1. Pegar o HTML parsear ele usando a bibliotea BeautifulSoup 
html_content = element.get_attribute('outerHTML')
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find(name='table')

# 7. Data Structure Conversion (Estruturar conteúdo em um Data Frame) - Pandas
df_full = pd.read_html(str(table))[0]

driver.quit()


## Tratando os dados

In [26]:
# 1. Excluindo as colunas vazias 
df_full = df_full.dropna(axis=1)

# 2. Renomeando os nomes das colunas 

df_full = df_full.rename(columns={
    'GP': 'Games_played',
    "W": 'Wins',
    "L": 'Losses',
    "Min": 'Minutes_played',
    "PTS": 'Points',
    "FGM": 'Field_goals_made',
    "FGA": 'Field_goals_attempted',
    "FG%": 'Field_goals_percentage',
    "3PM": '3_points_field_goals__made',
    "3PA": '3_point_field_goals_attempted',
    "3P%": '3_point_field_goals_percentage',
    "FTM": 'Free_throws_made',
    "FTA": 'Free_throws_attempted',
    "FT%": 'Free_throws_percentage',
    "OREB": 'Offensive_rebounds',
    "DREB": 'defensive_rebounds',
    
})
df_full = df_full.drop(columns=['Unnamed: 0'], axis=1)

dicionario = df_full.to_dict('records')


# Converter e salvar em um arquivo JSON
with open('dados.json', 'w', encoding='utf-8') as jp:
    js = json.dumps(dicionario, indent=4)
    jp.write(js)

## Subindo o Arquivo JSON em um Bucket AWS

In [ ]:
# 1. Inicializando o serviço S3 da AWS. 
s3 = boto3.resource(
    service_name='s3',
    region_name = 'sa-east-1',
    aws_access_key_id = key_id,
    aws_secret_access_key = access_key
)

# 2. Upload do arquivo desejado no Bucket 
s3.Bucket("webscraping2023").upload_file('dados.json',"dados.json")